# ***Organização Industrial - EAE0508***

In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import requests
import os
import trabalhodecurso

from pathlib import Path
from brfinance import CVMAsyncBackend
from datetime import datetime, date
from unidecode import unidecode
from bs4 import BeautifulSoup

### *Dados de Lojas*

In [17]:
#r = requests.get('https://www.marisa.com.br/store-finder')
#
#soup = BeautifulSoup(r.text, 'html.parser')
#
#if soup.find(id='storelocator-query') is not None:
#    print('Found')
#else: 
#    print('Not found.')

## *Análises*
***

In [72]:
demonstrativos = pd.read_excel('dados/demonstrativos.xlsx').sort_values(by='consolidado')
financas = pd.read_excel('dados/indicadores-financeiros.xlsx').sort_values(by='consolidado')
mercado = pd.read_excel('dados/indicadores-mercado.xlsx').sort_values(by='consolidado')
lojas = pd.read_excel('dados/lojas-e-funcionarios.xlsx').sort_values(by='Ano')
comercio = pd.read_excel('dados/pesquisa-comercio.xlsx')
acoes = pd.read_excel('dados/preco-acao.xlsx')

In [19]:
demonstrativos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333 entries, 230 to 332
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   consolidado               333 non-null    object 
 1   ativo total               333 non-null    float64
 2   ativo circulante          333 non-null    float64
 3   ativo nao circulante      333 non-null    float64
 4   imobilizado               333 non-null    float64
 5   passivo e patrimonio liq  333 non-null    float64
 6   passivo circulante        333 non-null    float64
 7   passivo nao circulante    333 non-null    float64
 8   patrim liq consolidado    333 non-null    float64
 9   lucros acumulados         328 non-null    float64
 10  +receita liquida operac   324 non-null    object 
 11  -custo produtos vendidos  325 non-null    object 
 12  lucro bruto               324 non-null    object 
 13  +despesas com vendas      325 non-null    object 
 14  lucro li

In [20]:
financas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333 entries, 230 to 332
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   consolidado               333 non-null    object 
 1   lpa lucro por acao $      333 non-null    object 
 2   vpa valor patr por acao$  333 non-null    object 
 3   vendas por acao $         333 non-null    object 
 4   ebitda por acao $         333 non-null    object 
 5   divida total liquida $    333 non-null    float64
 6   divida total bruta $      333 non-null    float64
 7   divida bruta / atv tt %   333 non-null    float64
 8   divida bruta / pat liq %  333 non-null    float64
 9   divida liq / pat liq %    333 non-null    float64
 10  estr cap (d/d+pat liq) %  333 non-null    float64
 11  ebit / divida bruta %     333 non-null    object 
 12  ebit / divida liquida %   333 non-null    object 
 13  ebit / desp fin brut x    333 non-null    object 
 14  ebit / d

In [21]:
mercado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409 entries, 204 to 408
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   consolidado               409 non-null    object
 1   meses                     409 non-null    int64 
 2   preco / lucro x           409 non-null    object
 3   preco / valor patrim x    409 non-null    object
 4   price sales ratio x       409 non-null    object
 5   preco / ebitda x          409 non-null    object
 6   preco / flx cx livre x    409 non-null    object
 7   lucro / preco %           409 non-null    object
 8   valor de mercado $        409 non-null    object
 9   enterprise value $        409 non-null    object
 10  estr cap (d/d+val merc)%  409 non-null    object
 11  ev / ebitda da empresa x  409 non-null    object
 12  ev / ebitda da classe  x  409 non-null    object
 13  ev / ebit da empresa   x  409 non-null    object
 14  ev / ebit da classe    x

### Indicadores de Performance

In [64]:
xaxis_attribs = dict(
    showline=True,
    showgrid=False,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    nticks=5,
    gridcolor="LightGrey",
)

yaxis_attribs = dict(
    showline=True,
    showgrid=True,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    nticks=10,
    gridcolor="LightGrey",
)

#### Margem Bruta 
***

In [56]:
empresas = ['Lojas Renner', 'Riachuelo', 'Marisa', 'C&A']
margem_bruta = financas.loc[(financas['consolidado'] >= '2010-01-01') 
                             & financas['empresa'].isin(empresas)]

fig = px.line(margem_bruta, 
              x='consolidado', 
              y='margem bruta %', 
              color='empresa', 
              height=800, 
              width=1000,
              markers=False)


xaxis_attribs = dict(showline=True, showgrid=False, tickfont=dict(family='Josefin Sans Bold'), gridcolor='LightGrey')
yaxis_attribs = dict(showline=True, showgrid=True, tickfont=dict(family='Josefin Sans Bold'), nticks=20, gridcolor='LightGrey')
legend_attribs = dict(yanchor='top', 
                      xanchor='left', 
                      y=0.99, 
                      x=0.01, 
                      title_font_family='Josefin Sans Bold', 
                      font=dict(family='Josefin Sans Bold', size=14))

fig.update_layout(
    xaxis_title = '',
    yaxis_title = 'Margem Bruta %',
    plot_bgcolor = 'white',
    legend_title_text='Empresa',
    xaxis = xaxis_attribs,
    yaxis = yaxis_attribs,
    legend = legend_attribs
)
fig.update_traces(line=dict(width=3))

fig.show()

#### Lucro Bruto
***

In [62]:
empresas = [
    "Lojas Renner",
    "Riachuelo",
    "Marisa",
    "C&A",
    "Arezzo",
    "Grupo SOMA",
    "Le Lis Blanc",
]
lucro_bruto = demonstrativos.loc[
    (demonstrativos["consolidado"] >= "2017-01-01")
    & demonstrativos["empresa"].isin(empresas)
]

lucro_bruto = lucro_bruto.assign(periodo=lucro_bruto["consolidado"].str.slice(0, 7))
lucro_bruto = lucro_bruto.pivot_table(
    index=["periodo"], columns="empresa", values="lucro bruto"
)
lucro_bruto = lucro_bruto.assign(total=lucro_bruto[empresas].sum(axis=1))

fig = px.line(
    lucro_bruto, x=lucro_bruto.index, y=empresas, height=800, width=1500, markers=False
)

legend_attribs = dict(
    yanchor="top",
    xanchor="left",
    y=0.99,
    x=0.01,
    title_font_family="Josefin Sans Bold",
    font=dict(family="Josefin Sans Bold", size=20),
)

fig.update_layout(
    xaxis_title="",
    yaxis_title="Lucro Bruto $",
    plot_bgcolor="white",
    legend_title_text="Empresa",
    xaxis=xaxis_attribs,
    yaxis=yaxis_attribs,
    legend=legend_attribs,
)
fig.update_traces(line=dict(width=3))

fig.show()

#### Número de Lojas
***

In [71]:
empresas = [
    "Lojas Renner",
    "Riachuelo",
    "Marisa",
    "C&A",
    "Arezzo",
    "Grupo SOMA",
    "Le Lis Blanc",
]
n_lojas = lojas.loc[(lojas["Companhia"].isin(empresas)) & (lojas["Ano"] > 2016)]

fig = px.line(
    n_lojas,
    x="Ano",
    y="N Lojas ",
    color="Companhia",
    height=800,
    width=1500,
    markers=False,
)

xaxis_attribs = dict(
    showline=True,
    showgrid=False,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    nticks=5,
    gridcolor="LightGrey",
)

legend_attribs = dict(
    yanchor="top",
    xanchor="left",
    y=0.99,
    x=0.01,
    title_font_family="Josefin Sans Bold",
    font=dict(family="Josefin Sans Bold", size=20),
)

fig.update_layout(
    xaxis_title="",
    yaxis_title="Número de Lojas",
    plot_bgcolor="white",
    legend_title_text="Empresa",
    xaxis=xaxis_attribs,
    yaxis=yaxis_attribs,
    legend=legend_attribs,
    font=dict(family="Josefin Sans Bold", size=20),
)
fig.update_traces(line=dict(width=3))

fig.show()

#### Prazo Médio de Estoque
***

In [68]:
empresas = [
    "Lojas Renner",
    "Riachuelo",
    "Marisa",
    "C&A",
    "Arezzo",
    "Grupo SOMA",
    "Le Lis Blanc",
]

estoques = financas.loc[
    (financas["consolidado"] >= "2010-01-01") & financas["empresa"].isin(empresas)
]

fig = px.line(
    estoques,
    x="consolidado",
    y="prazo med estoques(dias)",
    color="empresa",
    height=800,
    width=1000,
    markers=False,
)

xaxis_attribs = dict(
    showline=True,
    showgrid=False,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    gridcolor="LightGrey",
)

yaxis_attribs = dict(
    showline=True,
    showgrid=True,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    nticks=20,
    gridcolor="LightGrey",
)

legend_attribs = dict(
    yanchor="top",
    xanchor="left",
    y=0.99,
    x=0.01,
    title_font_family="Josefin Sans Bold",
    font=dict(family="Josefin Sans Bold", size=20),
)

fig.update_layout(
    xaxis_title="",
    yaxis_title="Prazo Médio de Estoques (dias)",
    plot_bgcolor="white",
    legend_title_text="Empresa",
    xaxis=xaxis_attribs,
    yaxis=yaxis_attribs,
    legend=legend_attribs,
)
fig.update_traces(line=dict(width=3))

fig.show()

#### Pesquisa Mensal do Comércio - Receita
***

In [51]:
empresas = [
    "Lojas Renner",
    "Riachuelo",
    "Marisa",
    "C&A",
    "Arezzo",
    "Grupo SOMA",
    "Le Lis Blanc",
]

tempo_relevante_comercio = comercio.loc[(comercio["periodo"] >= "2017-01-01")]

fig = px.line(
    tempo_relevante_comercio,
    x="periodo",
    y=["Receita - PMC - Total", "Receita - Tecidos, Vestuário e Calçados"],
    height=800,
    width=1500,
    markers=False,
)


xaxis_attribs = dict(
    showline=True,
    showgrid=False,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    gridcolor="LightGrey",
)
yaxis_attribs = dict(
    showline=True,
    showgrid=True,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    nticks=15,
    gridcolor="LightGrey",
)
legend_attribs = dict(
    yanchor="bottom",
    xanchor="left",
    y=0.05,
    x=0.01,
    title_font_family="Josefin Sans Bold",
    font=dict(family="Josefin Sans Bold", size=20),
)

fig.update_layout(
    xaxis_title="",
    yaxis_title="Receita Dessazonalizada",
    plot_bgcolor="white",
    legend_title_text="",
    xaxis=xaxis_attribs,
    yaxis=yaxis_attribs,
    legend=legend_attribs,
    font=dict(family="Josefin Sans Bold", size=20),
)
fig.update_traces(line=dict(width=3))

fig.show()

#### Preço das Ações
***

In [52]:
empresas = [
    "Lojas Renner",
    "Riachuelo",
    "Marisa",
    "C&A",
    "Arezzo",
    "Grupo SOMA",
    "Le Lis Blanc",
]

preco_acoes = acoes.loc[(acoes["periodo"] >= "2017-01-01")]

fig = px.line(
    preco_acoes, x="periodo", y=empresas, height=800, width=1500, markers=False
)


xaxis_attribs = dict(
    showline=True,
    showgrid=False,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    gridcolor="LightGrey",
)
yaxis_attribs = dict(
    showline=True,
    showgrid=True,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    nticks=15,
    gridcolor="LightGrey",
)
legend_attribs = dict(
    yanchor="top",
    xanchor="left",
    y=0.99,
    x=0.01,
    title_font_family="Josefin Sans Bold",
    font=dict(family="Josefin Sans Bold", size=20),
)

fig.update_layout(
    xaxis_title="",
    yaxis_title="Preço de Fechamento",
    plot_bgcolor="white",
    legend_title_text="",
    xaxis=xaxis_attribs,
    yaxis=yaxis_attribs,
    legend=legend_attribs,
    font=dict(family="Josefin Sans Bold", size=20),
)
fig.update_traces(line=dict(width=3))

fig.show()

#### Pesquisa Mensal do Comércio - Volume de Vendas
***

In [73]:
empresas = [
    "Lojas Renner",
    "Riachuelo",
    "Marisa",
    "C&A",
    "Arezzo",
    "Grupo SOMA",
    "Le Lis Blanc",
]

tempo_relevante_comercio = comercio.loc[(comercio["periodo"] >= "2017-01-01")]

fig = px.line(
    tempo_relevante_comercio,
    x="periodo",
    y=["Volume - PMC - Total", "Volume - Tecidos, Vestuário e Calçados"],
    height=800,
    width=1500,
    markers=False,
)
xaxis_attribs = dict(
    showline=True,
    showgrid=False,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    gridcolor="LightGrey",
)
yaxis_attribs = dict(
    showline=True,
    showgrid=True,
    tickfont=dict(family="Josefin Sans Bold", size=20),
    nticks=15,
    gridcolor="LightGrey",
)
legend_attribs = dict(
    yanchor="bottom",
    xanchor="left",
    y=0.05,
    x=0.01,
    title_font_family="Josefin Sans Bold",
    font=dict(family="Josefin Sans Bold", size=20),
)

fig.update_layout(
    xaxis_title="",
    yaxis_title="Volume de Vendas Dessazonalizado",
    plot_bgcolor="white",
    legend_title_text="",
    xaxis=xaxis_attribs,
    yaxis=yaxis_attribs,
    legend=legend_attribs,
    font=dict(family="Josefin Sans Bold", size=20),
)
fig.update_traces(line=dict(width=3))

fig.show()